In [1]:
!pip install geemap

In [2]:
import hydrofunctions as hf
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import folium
import geemap.foliumap as geemap
import ee
import matplotlib.dates as mdates
import hydrofns

%matplotlib inline

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWjdYItyzcMNH0uRRh_i8YsVykAHtVL-SIrJcHK4yqpof5mGaQVPHQ4

Successfully saved authorization token.


In [41]:
# site number for NWIS gage (Vicksburg MS Site)
vicksburg = ['07289000']

# Coordinates for the bounds of a rectangle at Vicksburg MS
vicksburg_xMin = -91.2
vicksburg_yMin = 32
vicksburg_xMax = -90.8
vicksburg_yMax = 32.7

# Vicksburg MS wet and dry seasons
vicksburg_wet_season_start = '2019-06-01'
vicksburg_wet_season_end = '2019-06-30'

vicksburg_dry_season_start = '2019-9-01'
vicksburg_dry_season_end = '2019-10-01'



In [42]:
# Import the Sentinel 2 collection as sentinel 2
sentinel2 = ee.ImageCollection(
    'COPERNICUS/S2_SR').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Vicksburg geometry
vicksburg_rectangleGeoJSON = ee.Geometry.Rectangle(
    [
        [vicksburg_xMin, vicksburg_yMin],
        [vicksburg_xMax, vicksburg_yMax]   # max x and y
    ]
)


vicksburg_bnds = ee.FeatureCollection([
    ee.Feature(
        vicksburg_rectangleGeoJSON,
        {'name': 'Mississippi Vicksburg', 'fill': 1}),
])

# Palettes for Visualization
trueColor_palette = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
waterPalette = ['red', 'yellow', 'green', 'blue']

In [43]:
# Import the Sentinel 2 collection as sentinel 2
vicksburg_dry_season = sentinel2.filterBounds(vicksburg_bnds).filterDate(
    vicksburg_dry_season_start, vicksburg_dry_season_end).mosaic().clip(vicksburg_bnds)
vicksburg_wet_season = sentinel2.filterBounds(vicksburg_bnds).filterDate(
    vicksburg_wet_season_start, vicksburg_wet_season_end).mosaic().clip(vicksburg_bnds)

In [44]:
Map = geemap.Map(center=[32.35, -90.9], zoom=10)
Map.addLayer(vicksburg_dry_season, trueColor_palette,
             '2019 Dry season true color')
Map.addLayerControl()
Map

In [45]:
Map = geemap.Map(center=[32.35, -90.9], zoom=10)
Map.addLayer(vicksburg_wet_season, trueColor_palette,
             '2019 Wet season true color')
Map.addLayerControl()
Map

In [24]:
vicksburg_dry_season_Map = geemap.Map(center=[32.35, -90.9], zoom=10)
vicksburg_ndwi_dry = vicksburg_dry_season.normalizedDifference(['B3', 'B8'])
vicksburg_dry_season_Map.addLayer(vicksburg_ndwi_dry, {
                                  'min': -1, 'max': 0.5, 'palette': waterPalette}, '2019 Wet season NDWI')
vicksburg_dry_season_Map

In [25]:
vicksburg_wet_season_Map = geemap.Map(center=[32.35, -90.9], zoom=10)
vicksburg_ndwi_wet = vicksburg_wet_season.normalizedDifference(['B3', 'B8'])
vicksburg_wet_season_Map.addLayer(vicksburg_ndwi_wet, {
                                  'min': -1, 'max': 0.5, 'palette': waterPalette}, '2019 Wet season NDWI')
vicksburg_wet_season_Map

In [31]:
vicksburg_dry_season_Map = geemap.Map(center=[32.35, -90.9], zoom=10)

ndwi_threshold = 0
vicksburg_dry_season_image = vicksburg_ndwi_dry.gt(ndwi_threshold).selfMask()

vicksburg_dry_season_Map.addLayer(vicksburg_dry_season_image, {
                                  'palette': 'blue'}, 'Water image')
vicksburg_dry_season_Map

In [30]:
vicksburg_wet_season_Map = geemap.Map(center=[32.35, -90.9], zoom=10)

ndwi_threshold = 0
vicksburg_wet_season_image = vicksburg_ndwi_wet.gt(ndwi_threshold).selfMask()

vicksburg_wet_season_Map.addLayer(vicksburg_wet_season_image, {
                                  'palette': 'blue'}, 'Water image')
vicksburg_wet_season_Map

In [32]:
# Reduce the region. The region parameter is the Feature geometry.
vicksburg_count_pixels_wet = vicksburg_wet_season_image.reduceRegion(**{
    'reducer': ee.Reducer.count(),
    'geometry': vicksburg_bnds.geometry(),
    'scale': 10,
    'maxPixels': 1e9
})

# Reduce the region. The region parameter is the Feature geometry.
vicksburg_count_pixels_dry = vicksburg_dry_season_image.reduceRegion(**{
    'reducer': ee.Reducer.count(),
    'geometry': vicksburg_bnds.geometry(),
    'scale': 10,
    'maxPixels': 1e9
})

# The result is a Dictionary.  Print it.
print("Vicksburg water pixel count in the wet season: ",
      vicksburg_count_pixels_wet.getInfo())
print("Vicksburg water pixel count in the dry season: ",
      vicksburg_count_pixels_dry.getInfo())

Vicksburg water pixel count in the wet season:  {'nd': 7168558}
Vicksburg water pixel count in the dry season:  {'nd': 2668216}


In [33]:
def get_water_pixel_count(bounds, start, end):
    """
    Add docstring here.
    """
    ndwi_threshold = 0
    sentinel_image = sentinel2.filterBounds(
        bounds).filterDate(start, end).mosaic().clip(bounds)
    ndwi_image = sentinel_image.normalizedDifference(['B3', 'B8'])
    water_pixel_image = ndwi_image.gt(ndwi_threshold).selfMask()

    pixel_count_dict = water_pixel_image.reduceRegion(**{
        'reducer': ee.Reducer.count(),
        'geometry': bounds.geometry(),
        'scale': 10,
        'maxPixels': 1e9})
    pixel_count = pixel_count_dict.getInfo()
    
    return pixel_count



In [47]:
# Need to figure out how to iterate through months with function
# Need to figure out how to iterate through months with function
vb_jan = get_water_pixel_count(vicksburg_bnds, '2019-01-01', '2019-01-30')
#vb_feb = get_water_pixel_count(vicksburg_bnds, '2019-02-01', '2019-02-28')
vb_mar = get_water_pixel_count(vicksburg_bnds, '2019-03-01', '2019-03-30')
vb_apr = get_water_pixel_count(vicksburg_bnds, '2019-04-01', '2019-04-30')
vb_may = get_water_pixel_count(vicksburg_bnds, '2019-05-01', '2019-05-30')
#vb_jun = get_water_pixel_count(vicksburg_bnds, '2019-06-01', '2019-06-30')
vb_jul = get_water_pixel_count(vicksburg_bnds, '2019-07-01', '2019-07-31')
vb_aug = get_water_pixel_count(vicksburg_bnds, '2019-08-01', '2019-08-30')
vb_sep = get_water_pixel_count(vicksburg_bnds, '2019-09-01', '2019-09-30')
vb_oct = get_water_pixel_count(vicksburg_bnds, '2019-10-01', '2019-10-30')
vb_nov = get_water_pixel_count(vicksburg_bnds, '2019-11-01', '2019-11-30')
vb_dec = get_water_pixel_count(vicksburg_bnds, '2019-12-01', '2019-12-31')
vb_feb = {'nd': ((vb_jan['nd'] + vb_mar['nd'])/2)}
vb_jun = {'nd': ((vb_may['nd'] + vb_jul['nd'])/2)}

list_pixels = [vb_jan, vb_feb, vb_mar, vb_apr, vb_may, vb_jun, vb_jul, vb_aug, vb_sep, vb_oct, vb_nov, vb_dec]

print(list_pixels)

[{'nd': 7306039}, {'nd': 8325239.0}, {'nd': 9344439}, {'nd': 7168540}, {'nd': 7087810}, {'nd': 6653620.0}, {'nd': 6219430}, {'nd': 2815882}, {'nd': 2668216}, {'nd': 2828022}, {'nd': 3054553}, {'nd': 48107}]
